# Handling of Resources in Vortex (bis)

In the first notebook/presentation, the way Vortex is "loaded" was not described. Let's be more talkative: since this document uses Jupyter notebooks and consequently IPython, we are using the IPython extension purposely develop for Vortex.

This extension is named *ivortex* and is located under the ``project`` sub-directory of the Vortex code package. It extends the usual IPython shell commands (the "magic" command ``%vortex tmpcocoon`` allows to load Vortex and to jump in a temporary directory that will be deleted when the notebook is stopped).

In [1]:
%load_ext ivortex
%vortex tmpcocoon

# [2019/08/27-18:10:19][vortex.sessions][_set_rundir:0152][WARNING]: Session <root> is changing its working directory </sauvegarde/vortex-dev/vortex/examples/notebooks/tutorial>
# [2019/08/27-18:10:19][vortex.sessions][_set_rundir:0155][INFO]: Session <root> set rundir </home/meunierlf/vortex-workdir/auto_cocoon_2meq6c56>


Vortex 1.6.2 loaded ( Tuesday 27. August 2019, at 18:10:18 )
The working directory is now: /home/meunierlf/vortex-workdir/auto_cocoon_2meq6c56/root


'/home/meunierlf/vortex-workdir/auto_cocoon_2meq6c56'

Note: A more detailed description is given in the documentation. See: [Notebooks Summary > Working with Vortex in IPython and Notebooks](http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/notebooks/00_ipython_and_notebooks_howto.html).

## Reminder of the previous example

In [2]:
rhandlers = toolbox.input(# Input function options
                          verbose=False,
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, geometry='euroc25',
                          nativefmt='grib', model='arpege', 
                          cutoff='production', date='2017060118',
                          origin='historic',
                          # Provider
                          vapp='arpege', vconf='pearp', member=2,
                          experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='my_pearp_gribfile', format='grib')
print(rhandlers)
rhandlers[0].clear()  # cleanup

True

It is now easier to understand the code behaviour: the **footprints** package is leveraged to create the *resource*, *provider* and *container* objects.

## Use of predefined values

Most of Vortex scripts (including Olive or operation's scripts) setup predefined values. Let's do the same thing here:

In [3]:
# The toolbox module provides a shortcut to the footprints' setup object
import bronx
toolbox.defaults(
    date = bronx.stdtypes.date.Date('2017060118'),
    cutoff = 'production',
    model = 'arpege',
    geometry = vortex.data.geometries.get(tag='euroc25'),
)
# It is also possible to prescribe a defaut for vapp and vconf 
t.glove.vapp = 'arpege'
t.glove.vconf = 'pearp'  # Given our knowledge of Vortex, it is too soon
                         # to explain this code bit

In addition, let's setup the **toolbox** module:

In [4]:
toolbox.active_now = True
toolbox.active_verbose = False

This will substantially simplify the _resource_'s _Handler_ definition:

In [5]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='my_pearp_gribfile', format='grib')
print(rhandlers)
rhandlers[0].clear()  # cleanup

# [2019/08/27-18:10:19][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: my_pearp_gribfile)
# [2019/08/27-18:10:19][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: my_pearp_gribfile)
# [2019/08/27-18:10:20][vortex.tools.net][get:0268][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


True

## Footprints' substitution mechanism (in action)

Frequently, some attributes are built using the values of other attributes... For example, we would like for the local GRIB file name to include information on the geographical domain, the term, the member's number, ...

This is made easy by footprints' substitution mechanism:

In [6]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry]_[member]+[term]', format='grib')
%ls
rhandlers[0].clear()  # Cleanup

# [2019/08/27-18:10:21][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_<vortex.data.geometries.LonlatGeometry | tag='euroc25' id='Large european BDAP target' area='EUROC25' r='02dg500'>_002+03:00)
# [2019/08/27-18:10:21][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_<vortex.data.geometries.LonlatGeometry | tag='euroc25' id='Large european BDAP target' area='EUROC25' r='02dg500'>_002+03:00)
# [2019/08/27-18:10:22][vortex.tools.net][get:0268][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


'GRIB_<vortex.data.geometries.LonlatGeometry | tag='\''euroc25'\'' id='\''Large european BDAP target'\'' area='\''EUROC25'\'' r='\''02dg500'\''>_002+03:00'


True

* The substitution mechanism is triggered, in characters strings, by the following syntax: ``[something]``

The result is quite deceiving:

* The formatting used for the geometry is not adequate at all;
* We cannot control the formatting of the term and member's number.

### Making use of the attribute's types

* The *geometry* is an object defined by a very specific Vortex class. Therefore, it is possible to access its attributes in order to obtain a more satisfactory name:


In [7]:
geo = vortex.data.geometries.Geometry('euroc25')
print("Geometry type:", type(geo))  # It depends on the geometry; globaln lon/lat, ...
print("The area covered by the geometry (as in Meteo-France's BDAP):", geo.area)

Geometry type: <class 'vortex.data.geometries.LonlatGeometry'>
The area covered by the geometry (as in Meteo-France's BDAP): EUROC25


* The *term* attribute is of type **bronx.stdtypes.date.Time**, so that we would like to use its attributes and properties to format the forecast term:

In [8]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member]+[term:fmthm]', format='grib')
%ls
rhandlers[0].clear()  # Cleanup

# [2019/08/27-18:10:22][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_002+0003:00)
# [2019/08/27-18:10:22][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_002+0003:00)
# [2019/08/27-18:10:22][vortex.tools.net][get:0268][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


GRIB_EUROC25_002+0003:00


True

### Integers formating

With integers (or real numbers), it is possible to specify a C-like formatting code:

In [9]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
rhandlers[0].clear()  # Cleanup

# [2019/08/27-18:10:22][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/08/27-18:10:22][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/08/27-18:10:22][vortex.tools.net][get:0268][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


GRIB_EUROC25_0002+0003:00


True

### Making use of more complex methods

For example, it is possible to perform calculation with a **Date** object. Here, we would like to add the forecast term to the date in order to obtain the validity date:

In [10]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[date:addterm_ymdh]_[member%04d]',
                          format='grib')
%ls
rhandlers[0].clear()  # Cleanup

# [2019/08/27-18:10:23][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_2017060121_0002)
# [2019/08/27-18:10:23][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_2017060121_0002)
# [2019/08/27-18:10:23][vortex.tools.net][get:0268][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


GRIB_EUROC25_2017060121_0002


True

## Footprints' expansion mechanism (in action)

Until now, we have retrieved data one by one. This could become tedious for tasks that requires a lot of input data.

The expansion mechanism, that takes place before footprints resolution, allows to create a large number of resource descriptions starting form a single concise description.

### List expansion

(hint: look at the *term* attribute)

In [11]:
rhandlers = toolbox.input(loglevel='warning',  # Decrease the logging verbosity
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=[0,3,6], origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
for rhandler in rhandlers:
    rhandler.clear()  # Cleanup

GRIB_EUROC25_0002+0000:00  GRIB_EUROC25_0002+0003:00  GRIB_EUROC25_0002+0006:00


### A fairly simple mechanism...

The "expansion" is performed by a unique **footprints**' utility function: **footprints.util.expand**. It accepts a single input dictionary and returns a list of dictionaries. With the previous example, it gives:

In [12]:
input_dict = dict(kind='gridpoint', term=[0,3,6], origin='historic', nativefmt='grib',
                  member=2, experiment='DBLE', namespace='vortex.archive.fr', block='forecast',
                  local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
for i, a_dict in enumerate(fp.util.expand(input_dict)):
    print(">>>Dictionaire #{:d}".format(i))
    print(a_dict)

>>>Dictionaire #0
{'term': 0, 'local': 'GRIB_[geometry:area]_[member%04d]+[term:fmthm]', 'kind': 'gridpoint', 'block': 'forecast', 'origin': 'historic', 'experiment': 'DBLE', 'namespace': 'vortex.archive.fr', 'format': 'grib', 'nativefmt': 'grib', 'member': 2}
>>>Dictionaire #1
{'term': 3, 'local': 'GRIB_[geometry:area]_[member%04d]+[term:fmthm]', 'kind': 'gridpoint', 'block': 'forecast', 'origin': 'historic', 'experiment': 'DBLE', 'namespace': 'vortex.archive.fr', 'format': 'grib', 'nativefmt': 'grib', 'member': 2}
>>>Dictionaire #2
{'term': 6, 'local': 'GRIB_[geometry:area]_[member%04d]+[term:fmthm]', 'kind': 'gridpoint', 'block': 'forecast', 'origin': 'historic', 'experiment': 'DBLE', 'namespace': 'vortex.archive.fr', 'format': 'grib', 'nativefmt': 'grib', 'member': 2}


As a consequence, using the expansion mechanism is entirely equivalent to manualy creating N sections via the functions of the **toolbox** module...

### Other example: lists expressed as characters string

(hint: look at the *term* attribute)

In [13]:
rhandlers = toolbox.input(loglevel='warning',  # Decrease the logging verbosity
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term='0,3,6', origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
for rhandler in rhandlers:
    rhandler.clear()  # Nettoyage

GRIB_EUROC25_0002+0000:00  GRIB_EUROC25_0002+0003:00  GRIB_EUROC25_0002+0006:00


### Other example: Rangex

(hint: look at the *term* attribute)

In [14]:
rhandlers = toolbox.input(loglevel='warning',  # Decrease the logging verbosity
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=fp.util.rangex('0-12-3'), origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
for rhandler in rhandlers:
    rhandler.clear()  # Nettoyage

GRIB_EUROC25_0002+0000:00  GRIB_EUROC25_0002+0006:00  GRIB_EUROC25_0002+0012:00
GRIB_EUROC25_0002+0003:00  GRIB_EUROC25_0002+0009:00


Here are some usage example of the *rangex* function:

In [15]:
print(fp.util.rangex(0, 12, 3))
print(fp.util.rangex('0-12-3', shift=24))

[0, 3, 6, 9, 12]
[24, 27, 30, 33, 36]


With hours and minutes:

In [16]:
print(fp.util.rangex('0:00', '3:00', '0:30'))

['0000:00', '0000:30', '0001:00', '0001:30', '0002:00', '0002:30', '0003:00']


With a more complex string:

In [17]:
print(fp.util.rangex('0-12-3,18-36-6,48'))

[0, 3, 6, 9, 12, 18, 24, 30, 36, 48]


### Globbing

The idea is to observe the content of the current directory to build variables by analysing the name of some of the files...

In [18]:
for tfile in ('f_euroc25_0001:00', 'f_euroc25_0002:00', 'f_glob25_0002:00', 'f_a_trap'):
    sh.touch(tfile)

In [19]:
rhandlers = toolbox.output(# Method/Section
                           now=False, role='GridpointData',
                           # Resource
                           geometry='[glob:g]', term='[glob:t]',
                           kind='gridpoint', origin='historic', nativefmt='grib',
                           # Provider
                           member=2, experiment='ABCD', namespace='vortex.archive.fr',
                           block='forecast',
                           # Container
                           local='f_{glob:g:\w+}_{glob:t:\d+:\d+}', format='grib')
for rhandler in rhandlers:
    print('From:', rhandler.container.localpath())
    print('To:  ', rhandler.locate())

From: f_euroc25_0001:00
To:   meunierlf@hendrix.meteo.fr:/home/m/marp/marp999/vortex/arpege/pearp/A/B/C/D/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0001:00.grib
From: f_euroc25_0002:00
To:   meunierlf@hendrix.meteo.fr:/home/m/marp/marp999/vortex/arpege/pearp/A/B/C/D/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0002:00.grib
From: f_glob25_0002:00
To:   meunierlf@hendrix.meteo.fr:/home/m/marp/marp999/vortex/arpege/pearp/A/B/C/D/20170601T1800P/mb002/forecast/grid.arpege-forecast.glob25+0002:00.grib


In [20]:
sh.rmall('f_*')

## A small representative example

In [21]:
rhandlers = toolbox.input(# Method/ection
                          loglevel='warning', role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          geometry='euroc25,glob05', term=fp.util.rangex('0-12-3'),
                          kind='gridpoint', origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
for rhandler in rhandlers:
    rhandler.clear()  # Cleanup

GRIB_EUROC25_0002+0000:00  GRIB_EUROC25_0002+0012:00  GRIB_GLOB05_0002+0009:00
GRIB_EUROC25_0002+0003:00  GRIB_GLOB05_0002+0000:00   GRIB_GLOB05_0002+0012:00
GRIB_EUROC25_0002+0006:00  GRIB_GLOB05_0002+0003:00
GRIB_EUROC25_0002+0009:00  GRIB_GLOB05_0002+0006:00



Any questions : *vortex.support@meteo.fr*